In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
# Sprint weekends
sprint_rounds = [2, 6, 13]

# Custom full-grid points mapping (1st=20 ... 20th=1)
custom_points_map = {pos: 21 - pos for pos in range(1, 21)}

# Store race-by-race results
all_rounds_data = []

for rnd in range(1, 15):
    # Load race session
    race = fastf1.get_session(2025, rnd, 'R')
    race.load(laps=False, telemetry=False, weather=False, messages=False)
    race_results = race.results[['DriverNumber', 'Abbreviation', 'Points', 'Position', 'Status']].copy()

    # Load sprint session if exists
    sprint_points = pd.Series(0, index=race_results.index, dtype=float)
    if rnd in sprint_rounds:
        sprint = fastf1.get_session(2025, rnd, 'S')
        sprint.load(laps=False, telemetry=False, weather=False, messages=False)
        sprint_results = sprint.results[['DriverNumber', 'Points']].set_index('DriverNumber')['Points']
        sprint_points = race_results['DriverNumber'].map(sprint_results).fillna(0)

    # Flag DNFs
    race_results['DNF'] = race_results['Status'] == 'Retired'

    # Actual points system total for the round (Race + Sprint)
    race_results['TotalPoints_Actual'] = race_results['Points'] + sprint_points

    # Custom points system for race only (no points if DNF)
    race_results['CustomRacePoints'] = race_results.apply(
        lambda row: custom_points_map.get(int(row['Position']), 0) if not row['DNF'] else 0, axis=1
    )
    # Add sprint points (unchanged)
    race_results['TotalPoints_Custom'] = race_results['CustomRacePoints'] + sprint_points

    race_results['Round'] = rnd
    all_rounds_data.append(race_results)

# Combine all rounds
results_df = pd.concat(all_rounds_data, ignore_index=True)

# Championship standings - actual
standings_actual = results_df.pivot_table(index='Round', columns='Abbreviation',
                                          values='TotalPoints_Actual', aggfunc='sum').fillna(0)
standings_actual = standings_actual.cumsum()

# Championship standings - custom
standings_custom = results_df.pivot_table(index='Round', columns='Abbreviation',
                                          values='TotalPoints_Custom', aggfunc='sum').fillna(0)
standings_custom = standings_custom.cumsum()

In [ ]:
# Function to add Round 0 with zero points
def add_round_zero(standings_df):
    zero_row = pd.DataFrame({col: [0] for col in standings_df.columns})
    zero_row.index = [0]  # round 0
    df_with_zero = pd.concat([zero_row, standings_df], ignore_index=True)
    df_with_zero.index.name = 'Round'
    return df_with_zero

# Add round 0 to both standings
standings_actual_zero = add_round_zero(standings_actual)
standings_custom_zero = add_round_zero(standings_custom)

# Create the smooth x-axis for animation
rounds_data = np.linspace(0, len(standings_actual_zero)-1, 701)

# Function to smooth standings for any number of drivers
def smooth_standings(standings_df):
    smooth_data = {'Round': np.round(rounds_data, 2)}
    for driver in standings_df.columns:
        smooth_data[driver] = np.round(
            np.interp(
                x=rounds_data,
                xp=range(0, len(standings_df)),  # includes round 0
                fp=standings_df[driver].values
            ),
            2
        )
    return pd.DataFrame(smooth_data)

# Smooth both actual and custom standings
smooth_standings_actual_df = smooth_standings(standings_actual_zero)
smooth_standings_custom_df = smooth_standings(standings_custom_zero)

In [ ]:
setup_plot(xyticksize=18, axeslabel=20)

# Example: choose which standings to animate
smooth_standings_df = smooth_standings_actual_df.copy()

# Get driver abbreviations (columns except 'Round')
drivers = ['PIA','NOR','RUS','VER','LEC','HAM','ANT','ALB'] # [col for col in smooth_standings_df.columns if col != 'Round']

fig, ax = plt.subplots(figsize=(14, 8))

# Create empty line & marker objects for each driver
lines = {}
marker_lines = {}
marker_labels = {}

for drv in drivers:

    # xdata = smooth_standings_df['Round'][:]
    # ydata = smooth_standings_df[drv][:]

    style = plotting.get_driver_style(identifier=drv, session=race, style=['color', 'linestyle'])
    
    line, = ax.plot([], [], label=drv, lw=3, **style)
    lines[drv] = line

    marker_line, = ax.plot([], [], lw=0, marker='o', color=style['color'], markersize=6)
    marker_lines[drv] = marker_line

    # Create text object for label (driver + points)
    label = ax.text(0, 0, "", fontsize=18, fontweight='bold',
                    color=style['color'], ha='left', va='center')
    marker_labels[drv] = label

# Axis labels and legend
ax.set_xticks(range(1, 15))
ax.set_xticklabels(ax.get_xticklabels())
ax.set_yticks(range(0,325,25))
ax.set_yticklabels(ax.get_yticklabels())

# Axis labels and title
ax.set_title('Driver Championship Points')
ax.set_xlabel('Round')
ax.set_ylabel('')

# Legend
# ax.legend(fontsize=12, loc='upper left')

# Update function for animation
def update(frame):
    ymin, ymax = 0, 25
    xmin, xmax = 0, 2

    for drv in drivers:
        xdata = smooth_standings_df['Round'][:frame]
        ydata = smooth_standings_df[drv][:frame]
        
        # Update main line
        lines[drv].set_data(xdata, ydata)

        # Markers only on integer rounds
        mask = xdata.isin(range(1, int(smooth_standings_df['Round'].max()) + 1))
        marker_lines[drv].set_data(xdata[mask], ydata[mask])

        # Update text label position and value
        if len(xdata) > 0:
            marker_labels[drv].set_position((xdata.iloc[-1] + 0.1, ydata.iloc[-1]))
            marker_labels[drv].set_text(f"{drv}")

        # Track axis limits
        if len(ydata) > 0:
            ymin = min(ymin, ydata.min())
            ymax = max(ymax, ydata.max())
        if len(xdata) > 0:
            xmin = min(xmin, xdata.min())
            xmax = max(xmax, xdata.max())

    # Add padding to y-axis
    # padding = (ymax - ymin) * 0.25 if ymax > ymin else 10
    ax.set_ylim(ymin, ymax + 25)
    ax.set_xlim(xmin, xmax + 2)

    return list(lines.values()) + list(marker_lines.values()) + list(marker_labels.values())

# Create animation
ani = FuncAnimation(
    fig,
    update,
    frames=len(smooth_standings_df) + 1,
    interval=10,
    blit=True,
    repeat=False
)

HTML(ani.to_jshtml())

In [ ]:
# Optional: Save animation
ani.save('./media/Reel9/driver_standings_actual.mp4', writer='ffmpeg', fps=100, dpi=300, bitrate=8000)

In [ ]:
setup_plot(xyticksize=18, axeslabel=20)

# Example: choose which standings to animate
smooth_standings_df = smooth_standings_custom_df.copy()

# Get driver abbreviations (columns except 'Round')
drivers = ['PIA','NOR','RUS','VER','LEC','HAM','ANT','ALB'] # [col for col in smooth_standings_df.columns if col != 'Round']

fig, ax = plt.subplots(figsize=(14, 8))

# Create empty line & marker objects for each driver
lines = {}
marker_lines = {}
marker_labels = {}

for drv in drivers:
    if drv == 'DOO':
        style = {'color': '#ff87bc', 'linestyle': 'dashed'}
    else:
        style = plotting.get_driver_style(identifier=drv, session=race, style=['color', 'linestyle'])
    
    line, = ax.plot([], [], label=drv, lw=3, **style)
    lines[drv] = line

    marker_line, = ax.plot([], [], lw=0, marker='o', color=style['color'], markersize=6)
    marker_lines[drv] = marker_line

    # Create text object for label (driver)
    label = ax.text(0, 0, "", fontsize=18, fontweight='bold',
                    color=style['color'], ha='left', va='center')
    marker_labels[drv] = label

# Axis labels and legend
ax.set_xticks(range(1, 15))
ax.set_xticklabels(ax.get_xticklabels())
ax.set_yticks(range(0,325,25))
ax.set_yticklabels(ax.get_yticklabels())

# Axis labels and title
ax.set_title('Driver Championship Points')
ax.set_xlabel('Round')
ax.set_ylabel('')

# # Legend
# ax.legend(fontsize=12, loc='upper left')

# Update function for animation
def update(frame):
    ymin, ymax = 0, 25
    xmin, xmax = 0, 2

    for drv in drivers:
        xdata = smooth_standings_df['Round'][:frame]
        ydata = smooth_standings_df[drv][:frame]
        
        # Update main line
        lines[drv].set_data(xdata, ydata)

        # Markers only on integer rounds
        mask = xdata.isin(range(1, int(smooth_standings_df['Round'].max()) + 1))
        marker_lines[drv].set_data(xdata[mask], ydata[mask])

        # Update text label position and value
        if len(xdata) > 0:
            marker_labels[drv].set_position((xdata.iloc[-1] + 0.1, ydata.iloc[-1]))
            marker_labels[drv].set_text(f"{drv}")

        # Track axis limits
        if len(ydata) > 0:
            ymin = min(ymin, ydata.min())
            ymax = max(ymax, ydata.max())
        if len(xdata) > 0:
            xmin = min(xmin, xdata.min())
            xmax = max(xmax, xdata.max())

    # Add padding to y-axis
    if ymin == float('inf'):
        ymin, ymax = 0, 10
    padding = (ymax - ymin) * 0.25 if ymax > ymin else 10
    ax.set_ylim(ymin, ymax + 25)
    ax.set_xlim(xmin, xmax + 2)

    return list(lines.values()) + list(marker_lines.values()) + list(marker_labels.values())

# Create animation
ani = FuncAnimation(
    fig,
    update,
    frames=len(smooth_standings_df) + 1,
    interval=10,
    blit=True,
    repeat=False
)

HTML(ani.to_jshtml())

In [ ]:
# Optional: Save animation
ani.save('./media/Reel9/driver_standings_custom.mp4', writer='ffmpeg', fps=100, dpi=300, bitrate=8000)